# Shunting Yard Algo

In [154]:
def insert_dot_before_alpha(string):
    result = ""
    found_first_alpha = False
    
    for char in string:
        if (char.isalpha() or char.isdigit()) and not found_first_alpha:
            result += char
            found_first_alpha = True
        elif (char.isalpha() or char.isdigit()):
            result += '.' + char
        else:
            result += char
            
    return result

def shuntingYardAlgo(infix):
    postfix = ""
    stack = []
    precedence = {'*': 5, '+': 4, '?': 3, '.': 2, '|': 1}
    opened=0
    infix=insert_dot_before_alpha(infix)
    print(infix)
    index = 0
    while index < len(infix):
        c = infix[index]
        if c == '(':
            stack.append(c)
            opened += 1
        elif c == ')':
            while stack[-1] != '(':
                postfix += stack.pop()
            if stack == []:
                return False
            stack.pop()
            opened -= 1
        elif c in precedence:
            while stack and precedence.get(stack[-1], 0) >= precedence[c]:
                postfix += stack.pop()
            stack.append(c)
        elif c == '[':
            transition = "[" 
            index += 1  
            while index < len(infix) and infix[index] != ']':
                if infix[index] != '.':
                    transition += infix[index]
                index += 1  
            if index >= len(infix):
                return False 
            transition += ']' 
           
            
            postfix += transition  
        else:
            postfix += c

        index += 1 


           
    if opened != 0:
        return False

    while stack:
        postfix += stack.pop()

    return postfix

    



# Postfix to NFA

In [162]:
import json
from graphviz import Digraph


class State:
    def __init__(self, label):
        self.label = label
        self.edges = []

class Edge:
    def __init__(self, value, destination):
        self.value = value
        self.destination = destination

class NFA:
    def __init__(self, initial, accept, states):
        self.initial = initial
        self.accept = accept
        self.states = states
    
    def to_json(self):
        fsm = {}
        fsm["startingState"] = self.initial.label
        
        for state in self.states:
            fsm[state.label] = {"isTerminatingState": state == self.accept}
            edges_sorted = sorted(state.edges, key=lambda x: x.destination.label)
            for edge in edges_sorted:
                if len(edges_sorted) == 1:
                    fsm[state.label][edge.value] = edge.destination.label
                else:
                    destinations = [edge.destination.label for edge in edges_sorted if edge.value == edge.value]
                    fsm[state.label][edge.value] = destinations
                    
        return fsm


def MakeState(counter, val, stack):
    state1 = State("S" + str(counter))
    state2 = State("S" + str(counter + 1))
    state1.edges.append(Edge(val, state2))
    nfa_made = NFA(state1, state2, [state1, state2])
    stack.append(nfa_made)

def zeroOrMore(counter, state, stack):
    nfa_made = stack.pop()
    state1 = State("S" + str(counter))
    state2 = State("S" + str(counter + 1))
    state1.edges.append(Edge('ε', nfa_made.initial))
    state1.edges.append(Edge('ε', state2))
    nfa_made.accept.edges.append(Edge('ε', state2))
    nfa_made.accept.edges.append(Edge('ε', state1))
    nfa_final = NFA(state1, state2, [state1, state2] + nfa_made.states)
    stack.append(nfa_final)

def oneOrMore(counter, state, stack):
    nfa_made = stack.pop()
    state1 = State("S" + str(counter))
    state2 = State("S" + str(counter + 1))
    state1.edges.append(Edge('ε', nfa_made.initial))
    nfa_made.accept.edges.append(Edge('ε', state2))
    nfa_made.accept.edges.append(Edge('ε', state1))
    nfa_final = NFA(state1, state2, [state1, state2] + nfa_made.states)
    stack.append(nfa_final)

def zeroOrOne(counter, state, stack):
    nfa_made = stack.pop()
    state1 = State("S" + str(counter))
    state2 = State("S" + str(counter + 1))
    state1.edges.append(Edge('ε', nfa_made.initial))
    state1.edges.append(Edge('ε', state2))
    nfa_made.accept.edges.append(Edge('ε', state2))
    nfa_final = NFA(state1, state2, [state1, state2] + nfa_made.states)
    stack.append(nfa_final)

def concatenate(counter, state, stack):
    nfa_made2 = stack.pop()
    nfa_made1 = stack.pop()
    nfa_made1.accept.edges.append(Edge('ε', nfa_made2.initial))
    nfa_final = NFA(nfa_made1.initial, nfa_made2.accept, nfa_made1.states + nfa_made2.states)
    stack.append(nfa_final)

def oring(counter, state, stack):
    nfa_made2 = stack.pop()
    nfa_made1 = stack.pop()
    state1 = State("S" + str(counter))
    state2 = State("S" + str(counter + 1))
    state1.edges.append(Edge('ε', nfa_made1.initial))
    state1.edges.append(Edge('ε', nfa_made2.initial))
    nfa_made1.accept.edges.append(Edge('ε', state2))
    nfa_made2.accept.edges.append(Edge('ε', state2))
    nfa_final = NFA(state1, state2, [state1, state2] + nfa_made1.states + nfa_made2.states)
    stack.append(nfa_final)

def thompsons(postfix):
    stack = []
    counter = 1
    alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    digits = '0123456789'
    index = 0
    while index < len(postfix):
        c = postfix[index]
        if c in alphabet or c in digits:
            MakeState(counter, c, stack)
            counter += 2
        elif c == '*':
            zeroOrMore(counter, c, stack)
            counter += 2
        elif c == '+':
            oneOrMore(counter, c, stack)
            counter += 2
        elif c == '?':
            zeroOrOne(counter, c, stack)
            counter += 2
        elif c == '.':
            concatenate(counter, c, stack)
        elif c == '|':
            oring(counter, c, stack)
            counter += 2
        elif c == '[':
            transition = "[" + postfix[index + 1] 
            index += 2  
            while postfix[index] != ']':
                transition += postfix[index]
                index += 1  
            transition += postfix[index] 
            
            MakeState(counter, transition, stack)
            counter += 2
        
        index += 1  

    return stack.pop()


infix = "a+b*"
postfix = shuntingYardAlgo(infix)
if(postfix==False):
    print("Invalid input")
else:
    print(postfix)
    nfa = thompsons(postfix)
    json_data = nfa.to_json()
    print(json.dumps(json_data, indent=4))


    graph = Digraph(graph_attr={'rankdir': 'LR'})
    graph.node('', shape='none')

    for key in json_data:
        if key != 'startingState':
            if json_data[key]["isTerminatingState"]:
                graph.node(name=key, label=key, shape='doublecircle')
            else:
                graph.node(name=key, label=key, shape='circle')

    for key in json_data:
        if key != 'startingState':
            for edge_value, destinations in json_data[key].items():
                if edge_value != 'isTerminatingState':
                    if isinstance(destinations, list):
                        for destination in destinations:
                            if edge_value == 'ε':
                                graph.edge(key, destination, label='ε')
                            else:
                                graph.edge(key, destination, label=edge_value)
                    else:
                        destination = destinations
                        if edge_value == 'ε':
                            graph.edge(key, destination, label='ε')
                        else:
                            graph.edge(key, destination, label=edge_value)

    graph.edge('', json_data['startingState'], label='')

    graph.render('./nfa1', view=True, format='png', cleanup=True)



a+.b*
a+b*.
{
    "startingState": "S3",
    "S3": {
        "isTerminatingState": false,
        "\u03b5": "S1"
    },
    "S4": {
        "isTerminatingState": false,
        "\u03b5": "S7"
    },
    "S1": {
        "isTerminatingState": false,
        "a": "S2"
    },
    "S2": {
        "isTerminatingState": false,
        "\u03b5": [
            "S3",
            "S4"
        ]
    },
    "S7": {
        "isTerminatingState": false,
        "\u03b5": [
            "S5",
            "S8"
        ]
    },
    "S8": {
        "isTerminatingState": true
    },
    "S5": {
        "isTerminatingState": false,
        "b": "S6"
    },
    "S6": {
        "isTerminatingState": false,
        "\u03b5": [
            "S7",
            "S8"
        ]
    }
}
